In [1]:
import os

from series_extraction.excel_loader import ExcelLoader
from series_extraction.excel_validator import ExcelValidator
from series_extraction.excel_cleaner import ExcelCleaner
from series_extraction.table_finder import TableFinder
from series_extraction.series_extractor import SeriesExtractor
from series_extraction.excel_compatibility_checker import ExcelCompatibilityChecker
from series_extraction.series_iterator import SeriesIterator
from series_extraction.series_mapper import SeriesMapper

from ast_building.formula_parser import FormulaParser
from ast_building.series_implementer import SeriesImplementer

from ast_transformation.formula_generator import FormulaGenerator
from ast_transformation.formula_checker import FormulaChecker

from excel_utils import ExcelUtils

In [2]:
current_directory = os.getcwd()

parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
data_directory = os.path.join(parent_directory, 'data')

project_name = 'test_excel_8'

excel_raw_file_path = os.path.join(data_directory, "excel_files_raw", f"{project_name}_raw.xlsx")
excel_reduced_filepath = os.path.join(data_directory, "excel_files_reduced", f"{project_name}_reduced.xlsx")

In [3]:
excel_raw= ExcelLoader.load_file(excel_raw_file_path)
excel_reduced = ExcelLoader.load_file(excel_reduced_filepath)

In [4]:
is_valid = ExcelValidator.validate_excel(excel_reduced)

In [5]:
if not is_valid:
    raise Exception("Excel file is not valid")

In [6]:
excel_reduced_clean = ExcelCleaner.clean_excel(excel_reduced)

In [7]:
extracted_tables, data = TableFinder.find_tables(excel_reduced_clean)

header values
['age', 'gender', 'time_spent', 'platform', 'interests', 'location', 'demographics', 'profession', 'income', 'indebt', 'isHomeOwner', 'Owns_Car', 'Index_Match_Size', 'If_Weight_Or_Size', 'Age Category', 'Average Time Spent for Gender', 'Interest Count', 'Platform Popularity', 'Gender code', 'Sumifs']
header values
['Unnamed: 0', 'Movie Name', 'Year of Release', 'Watch Time', 'Movie Rating', 'Meatscore of movie', 'Votes', 'Gross', 'Description', 'Lookup from Sheet1']


In [8]:
extracted_tables

{Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None): [Table(name='Sheet1_1', range=CellRange(start_cell=Cell(column=1, row=1, coordinate='A1', sheet_name=None, value=None, value_type=None), end_cell=Cell(column=20, row=3, coordinate='T3', sheet_name=None, value=None, value_type=None)), header_location=<HeaderLocation.TOP: 'top'>, header_values=['age', 'gender', 'time_spent', 'platform', 'interests', 'location', 'demographics', 'profession', 'income', 'indebt', 'isHomeOwner', 'Owns_Car', 'Index_Match_Size', 'If_Weight_Or_Size', 'Age Category', 'Average Time Spent for Gender', 'Interest Count', 'Platform Popularity', 'Gender code', 'Sumifs'])],
 Worksheet(sheet_name='Sheet2', workbook_file_path=None, worksheet=None): [Table(name='Sheet2_1', range=CellRange(start_cell=Cell(column=1, row=1, coordinate='A1', sheet_name=None, value=None, value_type=None), end_cell=Cell(column=10, row=3, coordinate='J3', sheet_name=None, value=None, value_type=None)), header_location=<Hea

In [9]:
series_data = SeriesExtractor.extract_table_details(extracted_tables, data)

In [10]:
is_compatible = ExcelCompatibilityChecker.check_file(excel_raw, excel_reduced, extracted_tables)

In [11]:
if not is_compatible:
    raise Exception("Excel file is not compatible")

In [12]:
series_dict = SeriesExtractor.extract_series(extracted_tables=extracted_tables, data=data)

In [13]:
series_dict

{'Sheet1': [Series(series_id=SeriesId(sheet_name='Sheet1', series_header='age', series_header_cell_row=1, series_header_cell_column=1), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='age', formulas=[None, None], values=[56, 46], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=1, row=2, coordinate='A2', sheet_name=None, value=None, value_type=None), series_length=2, series_data_type=<SeriesDataType.INT: 'int'>),
  Series(series_id=SeriesId(sheet_name='Sheet1', series_header='gender', series_header_cell_row=1, series_header_cell_column=2), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='gender', formulas=[None, None], values=['male', 'female'], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=2, row=2, coordinate='B2', sheet_name=None, value=None, value_type=None), series_length=2, series_data_type=<SeriesDataType.STR: 'str'>),
  

In [14]:
series_mapping = SeriesMapper.map_series(series_dict)

In [15]:
series_mapping

{Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None): {Cell(column=1, row=2, coordinate=None, sheet_name=None, value=None, value_type=None): (0,
   Series(series_id=SeriesId(sheet_name='Sheet1', series_header='age', series_header_cell_row=1, series_header_cell_column=1), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='age', formulas=[None, None], values=[56, 46], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=1, row=2, coordinate='A2', sheet_name=None, value=None, value_type=None), series_length=2, series_data_type=<SeriesDataType.INT: 'int'>)),
  Cell(column=1, row=3, coordinate=None, sheet_name=None, value=None, value_type=None): (1,
   Series(series_id=SeriesId(sheet_name='Sheet1', series_header='age', series_header_cell_row=1, series_header_cell_column=1), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='age', formulas=[None, None], v

In [16]:
series_iterator = SeriesIterator.iterate_series(series_dict)

In [17]:
series_list = [series for series in series_iterator]

# Create a list of series that have formulas
series_list_with_formulas = [series for series in series_list if series.formulas != [None, None]]

# Create a list of series that do not have formulas
series_list_with_values = [series for series in series_list if series.formulas == [None, None]]

In [18]:
series_list_with_formulas

[Series(series_id=SeriesId(sheet_name='Sheet1', series_header='Index_Match_Size', series_header_cell_row=1, series_header_cell_column=13), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='Index_Match_Size', formulas=['=INDEX(B:B,MATCH(A2,A:A,0))', '=INDEX(B:B,MATCH(A3,A:A,0))'], values=['male', 'female'], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=13, row=2, coordinate='M2', sheet_name=None, value=None, value_type=None), series_length=2, series_data_type=<SeriesDataType.STR: 'str'>),
 Series(series_id=SeriesId(sheet_name='Sheet1', series_header='If_Weight_Or_Size', series_header_cell_row=1, series_header_cell_column=14), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='If_Weight_Or_Size', formulas=['=IF(OR(B2>0,C2>0),"Acceptable","Check")', '=IF(OR(B3>0,C3>0),"Acceptable","Check")'], values=['Acceptable', 'Acceptable'], header_location=<HeaderLocation.TOP

In [19]:
series_list_with_values

[Series(series_id=SeriesId(sheet_name='Sheet1', series_header='age', series_header_cell_row=1, series_header_cell_column=1), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='age', formulas=[None, None], values=[56, 46], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=1, row=2, coordinate='A2', sheet_name=None, value=None, value_type=None), series_length=2, series_data_type=<SeriesDataType.INT: 'int'>),
 Series(series_id=SeriesId(sheet_name='Sheet1', series_header='gender', series_header_cell_row=1, series_header_cell_column=2), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='gender', formulas=[None, None], values=['male', 'female'], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=2, row=2, coordinate='B2', sheet_name=None, value=None, value_type=None), series_length=2, series_data_type=<SeriesDataType.STR: 'str'>),
 Series(series

In [20]:
import xlcalculator
import ast

from objects import Cell, HeaderLocation, CellRange, Column, CellRangeColumn

from excel_utils import ExcelUtils
from ast_transformation.formula_generator import SeriesIdLoader

class CellRangeImplementer:

    def merge_cell_ranges(self, cell_ranges):

        min_row = min(cell_range.start_cell.row for cell_range in cell_ranges)
        min_column = min(cell_range.start_cell.column for cell_range in cell_ranges)
        max_row = max(cell_range.end_cell.row for cell_range in cell_ranges)
        max_column = max(cell_range.end_cell.column for cell_range in cell_ranges)

        start_cell = Cell(column=min_column, row=min_row, coordinate = ExcelUtils.get_coordinate_from_column_and_row(min_column, min_row), sheet_name=cell_ranges[0].start_cell.sheet_name)
        end_cell = Cell(column=max_column, row=max_row, coordinate = ExcelUtils.get_coordinate_from_column_and_row(max_column, max_row), sheet_name=cell_ranges[0].start_cell.sheet_name)

        return CellRange(start_cell=start_cell, end_cell=end_cell)

    def create_cell_range_top_header(self, start_index, end_index, cell_row, cell_column, sheet_name):

        return CellRange(
            start_cell=Cell(row=cell_row+start_index, column=cell_column, coordinate=ExcelUtils.get_coordinate_from_column_and_row(cell_column, cell_row+start_index), sheet_name=sheet_name),
            end_cell=Cell(row=cell_row+ start_index, column=cell_column, coordinate=ExcelUtils.get_coordinate_from_column_and_row(cell_column, cell_row+start_index), sheet_name=sheet_name)
        )

    def create_cell_range_left_header(self, start_index, end_index, cell_row, cell_column, sheet_name):
        return CellRange(
            start_cell=Cell(row=cell_row, column=cell_column+start_index, coordinate=ExcelUtils.get_coordinate_from_column_and_row(cell_column+start_index, cell_row), sheet_name=sheet_name),
            end_cell=Cell(row=cell_row, column=cell_column+start_index, coordinate=ExcelUtils.get_coordinate_from_column_and_row(cell_column+start_index, cell_row), sheet_name=sheet_name)
        )
    
    def get_cell_range_from_series_tuple(self, series_tuple):
        
        series_ids_string, indexes = series_tuple
        series_start_index, series_end_index = indexes

        if series_start_index is None and series_end_index is None:
            return self.process_series_columns(series_ids_string)
        
        return self.process_series_cells(series_ids_string, series_start_index, series_end_index)

    def process_series_columns(self, series_ids_string):
        column_values = []

        sheet_name = SeriesIdLoader.load_series_id_from_string(series_ids_string[0]).sheet_name
        for series_id_string in series_ids_string:
            column_value = self.get_column_from_series_id(series_id_string)
            column_values.append(column_value)

        sorted_column_values = sorted(column_values, key=lambda x: x.column_number)
        return CellRangeColumn(start_column=sorted_column_values[0], end_column=sorted_column_values[-1], sheet_name=sheet_name)

    def get_column_from_series_id(self, series_id_string):
        series_id = SeriesIdLoader.load_series_id_from_string(series_id_string)
        sheet_name = series_id.sheet_name
        series_list = series_dict.get(sheet_name)

        for series in series_list:
            if series.series_id == series_id:
                column_value = series.series_starting_cell.column
                return Column(column_number=column_value, column_letter=ExcelUtils.get_column_letter_from_number(column_value))

    def process_series_cells(self, series_ids_string, series_start_index, series_end_index):

        cell_ranges = []

        for series_id_string in series_ids_string:

            cell_range = self.get_cell_range_for_series_id(series_id_string, series_start_index, series_end_index)
            cell_ranges.append(cell_range)

        return self.merge_cell_ranges(cell_ranges)

    def get_cell_range_for_series_id(self, series_id_string, series_start_index, series_end_index):
        series_id = SeriesIdLoader.load_series_id_from_string(series_id_string)
        sheet_name = series_id.sheet_name
        series_list = series_dict.get(sheet_name)

        for series in series_list:
            if series.series_id == series_id:
                return self.create_cell_range(series, series_start_index, series_end_index, sheet_name)

    def create_cell_range(self, series, series_start_index, series_end_index, sheet_name):
        cell_value = series.series_starting_cell
        cell_row = cell_value.row
        cell_column = cell_value.column

        if series.header_location == HeaderLocation.TOP:
            return self.create_cell_range_top_header(series_start_index, series_end_index, cell_row, cell_column, sheet_name)
        elif series.header_location == HeaderLocation.LEFT:
            return self. create_cell_range_left_header(series_start_index, series_end_index, cell_row, cell_column, sheet_name)
        else:
            raise Exception("Header location is not valid")

    def update_ast(self, ast):
        if isinstance(ast, xlcalculator.ast_nodes.RangeNode):
            return self.replace_range_node(ast)
        elif isinstance(ast, xlcalculator.ast_nodes.FunctionNode):
            return self.replace_function_node(ast)
        elif isinstance(ast, xlcalculator.ast_nodes.OperatorNode):
            return self.replace_operator_node(ast)
        return ast

    def replace_range_node(self, node):

        series_tuple = ast.literal_eval(node.tvalue)
        cell_range = self.get_cell_range_from_series_tuple(series_tuple)

        return xlcalculator.ast_nodes.RangeNode(
            xlcalculator.tokenizer.f_token(
                tvalue=cell_range, ttype="operand", tsubtype="range"
            ))

    def replace_function_node(self, node):
        modified_args = [self.update_ast(arg) for arg in node.args]
        modified_function_node = xlcalculator.ast_nodes.FunctionNode(node.token)
        modified_function_node.args = modified_args
        return modified_function_node

    def replace_operator_node(self, node):
        modified_left = self.update_ast(node.left) if node.left else None
        modified_right = self.update_ast(node.right) if node.right else None
        modified_operator_node = xlcalculator.ast_nodes.OperatorNode(node.token)
        modified_operator_node.left = modified_left
        modified_operator_node.right = modified_right
        return modified_operator_node

In [21]:
series_list_with_formulas

[Series(series_id=SeriesId(sheet_name='Sheet1', series_header='Index_Match_Size', series_header_cell_row=1, series_header_cell_column=13), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='Index_Match_Size', formulas=['=INDEX(B:B,MATCH(A2,A:A,0))', '=INDEX(B:B,MATCH(A3,A:A,0))'], values=['male', 'female'], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=13, row=2, coordinate='M2', sheet_name=None, value=None, value_type=None), series_length=2, series_data_type=<SeriesDataType.STR: 'str'>),
 Series(series_id=SeriesId(sheet_name='Sheet1', series_header='If_Weight_Or_Size', series_header_cell_row=1, series_header_cell_column=14), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='If_Weight_Or_Size', formulas=['=IF(OR(B2>0,C2>0),"Acceptable","Check")', '=IF(OR(B3>0,C3>0),"Acceptable","Check")'], values=['Acceptable', 'Acceptable'], header_location=<HeaderLocation.TOP

In [22]:
series_list_new = []

for series in series_list_with_formulas:
    formula_1 = series.formulas[0]
    formula_1_ast = FormulaParser.parse_formula(formula_1)
    series_implementer = SeriesImplementer(series_mapping, sheet_name = series.worksheet.sheet_name)
    formula_1_ast_series = series_implementer.update_ast(formula_1_ast)

    formula_2 = series.formulas[1]
    formula_2_ast = FormulaParser.parse_formula(formula_2)
    formula_2_ast_series = series_implementer.update_ast(formula_2_ast)
    
    sheet_name = series.worksheet.sheet_name

    series_list = series_dict.get(sheet_name)

    ast_generator = FormulaGenerator.get_ast_generator(formula_1_ast_series, formula_2_ast_series, series_list)

    formulas_are_correct, formula_1_ast_new, formula_2_ast_new = FormulaChecker.check_formulas(ast_generator)

    cell_range_implementer = CellRangeImplementer()


    formula_1_ast_new_cell_ranges = cell_range_implementer.update_ast(formula_1_ast_new)
    formula_2_ast_new_cell_ranges = cell_range_implementer.update_ast(formula_2_ast_new)
    
    series.formulas = [f"={formula_1_ast_new_cell_ranges}", f"={formula_2_ast_new_cell_ranges}"]
    series_list_new.append(series)
    
    if not formulas_are_correct:
        raise Exception("Formulas are not correct")

sheet_name inside
Sheet1
node.tvalue inside
B:B
cell_range inside
B:B
series_range inside
SeriesRange(series=[Series(series_id=SeriesId(sheet_name='Sheet1', series_header='gender', series_header_cell_row=1, series_header_cell_column=2), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='gender', formulas=[None, None], values=['male', 'female'], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=2, row=2, coordinate='B2', sheet_name=None, value=None, value_type=None), series_length=2, series_data_type=<SeriesDataType.STR: 'str'>), Series(series_id=SeriesId(sheet_name='Sheet1', series_header='gender', series_header_cell_row=1, series_header_cell_column=2), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='gender', formulas=[None, None], values=['male', 'female'], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=2, row=2, coordinate='B2', s

In [23]:
series_list_new = series_list_new + series_list_with_values

In [24]:
series_list_new

[Series(series_id=SeriesId(sheet_name='Sheet1', series_header='Index_Match_Size', series_header_cell_row=1, series_header_cell_column=13), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='Index_Match_Size', formulas=['=INDEX(Sheet1!B:B, MATCH(Sheet1!A2, Sheet1!A:A, 0))', '=INDEX(Sheet1!B:B, MATCH(Sheet1!A3, Sheet1!A:A, 0))'], values=['male', 'female'], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=13, row=2, coordinate='M2', sheet_name=None, value=None, value_type=None), series_length=2, series_data_type=<SeriesDataType.STR: 'str'>),
 Series(series_id=SeriesId(sheet_name='Sheet1', series_header='If_Weight_Or_Size', series_header_cell_row=1, series_header_cell_column=14), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='If_Weight_Or_Size', formulas=['=IF(OR((Sheet1!B2) > (0), (Sheet1!C2) > (0)), "Acceptable", "Check")', '=IF(OR((Sheet1!B3) > (0), (Sheet1!C3) 

In [25]:
from openpyxl import Workbook
from openpyxl.utils import get_column_letter

def create_excel_from_series(series_list):
    # Initialize a new workbook and dictionary to track existing sheets
    wb = Workbook()
    wb.remove(wb.active)  # Start with a clean slate by removing the default sheet
    ws_dict = {}

    # Iterate through each series object
    for series in series_list:
        sheet_name = series.series_id.sheet_name
        series_header = series.series_header
        header_row = series.series_id.series_header_cell_row
        header_col = series.series_id.series_header_cell_column
        header_location = series.header_location
        formulas = series.formulas
        values = series.values
        start_row = series.series_starting_cell.row
        start_col = series.series_starting_cell.column

        # Ensure the worksheet exists or create it if not
        if sheet_name not in ws_dict:
            if sheet_name in wb.sheetnames:
                ws = wb[sheet_name]
            else:
                ws = wb.create_sheet(title=sheet_name)
            ws_dict[sheet_name] = ws
        ws = ws_dict[sheet_name]

        # Place the series header
        ws.cell(row=header_row, column=header_col, value=series_header)

        # Fill the cells with formulas or values
        for i in range(series.series_length):

            row = start_row + i if header_location.value == "top" else start_row
            col = start_col + i if header_location.value == "left" else start_col
            
            if formulas != [None, None]:
                cell = ws.cell(row=row, column=col)
                cell.value = formulas[i]
            else:
                cell = ws.cell(row=row, column=col)
                cell.value = values[i]

    # Save the workbook to a file
    wb.save('output_series.xlsx')

# Example usage:
create_excel_from_series(series_list_new)


Series(series_id=SeriesId(sheet_name='Sheet1', series_header='Index_Match_Size', series_header_cell_row=1, series_header_cell_column=13), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='Index_Match_Size', formulas=['=INDEX(Sheet1!B:B, MATCH(Sheet1!A2, Sheet1!A:A, 0))', '=INDEX(Sheet1!B:B, MATCH(Sheet1!A3, Sheet1!A:A, 0))'], values=['male', 'female'], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=13, row=2, coordinate='M2', sheet_name=None, value=None, value_type=None), series_length=2, series_data_type=<SeriesDataType.STR: 'str'>)
['=INDEX(Sheet1!B:B, MATCH(Sheet1!A2, Sheet1!A:A, 0))', '=INDEX(Sheet1!B:B, MATCH(Sheet1!A3, Sheet1!A:A, 0))']
['male', 'female']
Series(series_id=SeriesId(sheet_name='Sheet1', series_header='Index_Match_Size', series_header_cell_row=1, series_header_cell_column=13), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='Index_Match_Siz